# IMPORT

In [1]:
# Importing the required libraries
import torch # Import the Pytorch library
import torchvision # Import the torchvision library
from torchvision import transforms # Import the transforms module from torchvision

from PIL import Image # Import the Image module from the Python Imaging Library (PIL)

import urllib # Import the urllib library for URL handling

# PRETRAINED MODEL IMPLEMENTATION

In [2]:
# Loading the pre-trained Inception v3 model from the Pytorch hub
# The 'pytorch/vision:v0.10.0' is the repository where the model is located
# 'inception_v3' is the name of the model
# The 'pretrained=True' flag loads the pre-trained weights
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)

# Setting the model to evaluation mode
model.eval()

Using cache found in /Users/fritt/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

### EXAMPLE

In [3]:
# Download an example image from the pytorch website

# URL of the image to be downloaded
url = "https://github.com/pytorch/hub/raw/master/images/dog.jpg"

# Name to be given to the downloaded image file
filename = "local_files/dog.jpg"

# Try to download the image using the urllib.URLopener() method (for Python 2)
try:
    urllib.URLopener().retrieve(url, filename)

# If the above method fails, use the urllib.request.urlretrieve() method (for Python 3)
except:
    urllib.request.urlretrieve(url, filename)

In [4]:
# Open the downloaded image file
input_image = Image.open(filename)

# Define image pre-processing steps
preprocess = transforms.Compose([
    transforms.Resize(299), # Resize the image to 299x299 pixels
    transforms.CenterCrop(299), # Crop the image to 299x299 pixels (removing any extra pixels)
    transforms.ToTensor(), # Convert the image to a Pytorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image using the pre-trained model's mean and standard deviation
])

# Apply the pre-processing steps to the image
input_tensor = preprocess(input_image)

# Add a batch dimension to the image (required by the model)
input_batch = input_tensor.unsqueeze(0)

# Move the input and model to GPU for faster processing if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# Disable gradient computation (not needed for inference)
with torch.no_grad():
    # Use the model to classify the image
    output = model(input_batch)

# The output has unnormalized scores for the 1000 ImageNet classes
print(output[0])

# To get the class probabilities, apply a softmax function on the output
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Print the class probabilities
print(probabilities)


tensor([ 1.5696e-01,  1.1051e-01, -4.4363e-01, -6.1476e-02, -2.0093e-01,
         1.6820e-01,  5.8117e-01,  6.0332e-02, -7.7653e-02, -1.1192e+00,
        -2.1800e-01, -5.2624e-01, -1.1816e-01,  6.6276e-02,  8.4988e-01,
        -5.3259e-02, -4.7113e-01, -2.5993e-03,  2.7504e-01,  1.7060e-01,
         4.4231e-01, -3.8615e-01,  1.3482e-01, -5.2202e-01, -3.3703e-04,
        -2.3492e-01, -2.3264e-01, -7.1883e-03,  1.6899e-02, -1.2724e-01,
         2.0508e-01, -6.9887e-03,  1.3074e+00, -2.3181e-01,  9.8966e-01,
        -7.7782e-01,  4.8900e-01, -4.2244e-01, -2.6523e-01, -1.5005e-01,
         3.5147e-01, -1.7462e-01,  1.0251e-01,  4.2223e-01, -1.4342e-01,
         1.3578e-02, -4.6477e-01,  3.5966e-01,  1.5194e-01,  2.0217e-01,
        -1.3763e-01,  1.4986e-01,  2.8415e-01, -3.7086e-01,  9.7689e-01,
         7.9543e-01, -7.4033e-01, -7.8853e-02, -9.6366e-02,  1.2193e-01,
        -1.2712e-01,  3.6825e-01, -4.5355e-01, -7.4801e-01, -7.9356e-02,
         2.1688e-01,  2.7018e-01,  1.0089e+00,  4.0

In [5]:
# Download ImageNet labels
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [7]:
# Read the categories from a text file
with open("local_files/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Get the top 5 categories with the highest probabilities
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Print the top 5 categories and their probabilities
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8238477110862732
Arctic fox 0.014191544614732265
white wolf 0.012303504161536694
Pomeranian 0.007874640636146069
keeshond 0.006387791130691767
